# Hector Adapter

In [193]:
# Import Packages
import logging
import os.path

from configparser import ConfigParser
import matplotlib.pyplot as plt
import pandas as pd
import scmdata

import openscm_runner
from openscm_runner.run import run
from openscm_runner.adapters import HECTOR
from openscm_runner.adapters import CICEROSCM

In [3]:
# Read in Future input emissions
input_emissions = scmdata.ScmRun(
    os.path.join(
        "..", "tests", "test-data", "rcmip_scen_ssp_world_emissions.csv"
    ),
    lowercase_cols=True,
)

input_emissions.head(30)

/Users/prim232/gitrepos/openscm-runner/venv/lib/python3.11/site-packages/scmdata/run.py:905: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  df.columns = pd.Index(columns, name="time")


time                                                                                                    2015-01-01  \
model   region scenario                  unit             variable                                                   
AIM/CGE World  ssp370                    Mt BC/yr         Emissions|BC                                    9.727424   
                                         kt C2F6/yr       Emissions|C2F6                                  1.570000   
                                         kt C6F14/yr      Emissions|C6F14                                 0.350000   
                                         kt CF4/yr        Emissions|CF4                                  10.869900   
                                         Mt CH4/yr        Emissions|CH4                                 388.072796   
                                         Mt CO/yr         Emissions|CO                                  934.349885   
                                         Mt CO2/yr        Emissions|CO2|MAGICC AFOLU                   3517.440000   
                                                          Emissions|CO2|MAGICC Fossil and Industrial  35635.286298   
                                         kt HFC125/yr     Emissions|HFC125                               78.790600   
                                         kt HFC134a/yr    Emissions|HFC134a                             201.953400   
                                         kt HFC143a/yr    Emissions|HFC143a                              32.381000   
                                         kt HFC227ea/yr   Emissions|HFC227ea                              3.663300   
                                         kt HFC23/yr      Emissions|HFC23                                14.485700   
                                         kt HFC245fa/yr   Emissions|HFC245fa                             11.728500   
                                         kt HFC32/yr      Emissions|HFC32                                38.579200   
                                         kt HFC4310mee/yr Emissions|HFC4310mee                            1.151600   
                                         kt N2O/yr        Emissions|N2O                               10900.000000   
                                         Mt NH3/yr        Emissions|NH3                                  65.279703   
                                         Mt NOx/yr        Emissions|NOx                                 155.520450   
                                         Mt OC/yr         Emissions|OC                                   34.746024   
                                         kt SF6/yr        Emissions|SF6                                   8.020000   
                                         Mt SO2/yr        Emissions|Sulfur                              100.771167   
                                         Mt VOC/yr        Emissions|VOC                                 227.244819   
               ssp370-lowNTCF-aerchemmip Mt BC/yr         Emissions|BC                                    9.727424   
                                         kt C2F6/yr       Emissions|C2F6                                  1.570000   
                                         kt C6F14/yr      Emissions|C6F14                                 0.350000   
                                         kt CF4/yr        Emissions|CF4                                  10.869900   
                                         Mt CH4/yr        Emissions|CH4                                 388.072796   
                                         Mt CO/yr         Emissions|CO                                  934.349885   
                                         Mt CO2/yr        Emissions|CO2|MAGICC AFOLU                   3517.440000   

time                                                                                                    2020-01-01  \
model   region scenario                  unit             variable                                       

In [156]:
# Set Hector configs
hector_cfgs = {
    'beta': 0.60,
    'S': 3.5,
    'diff': 1.2
}

In [5]:
# Output Variables
output_variables=(
        "Surface Air Temperature Change",
        "Atmospheric Concentrations|CO2",
    )

In [7]:
hector = HECTOR()

# Breakdown of Hector Run Function
The following will be the implementation of `hector._run()` broken out explicitly. The goal will be for this functionality to be called like
```
hector._run(
    scenarios=input_emissions,
    cfgs=hector_cfgs,
    output_variables=output_variables,
    output_config=None
)
```

In [252]:
cfgs = hector_cfgs
scenarios = input_emissions
runs_input = [
            {"cfgs": cfgs, "output_variables": output_variables, "scenariodata": smdf}
            for (scen, model), smdf in scenarios.timeseries().groupby(["scenario", "model"])
        ]
runs_input[0]

/Users/prim232/gitrepos/openscm-runner/venv/lib/python3.11/site-packages/scmdata/run.py:905: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  df.columns = pd.Index(columns, name="time")


{'cfgs': {'beta': 0.6, 'S': 3.5, 'diff': 1.2},
 'output_variables': ('Surface Air Temperature Change',
  'Atmospheric Concentrations|CO2'),
 'scenariodata': time                                                                                 2015-01-01  \
 model region scenario unit             variable                                                   
 IMAGE World  ssp119   Mt BC/yr         Emissions|BC                                    9.727424   
                       kt C2F6/yr       Emissions|C2F6                                  1.570000   
                       kt C6F14/yr      Emissions|C6F14                                 0.350000   
                       kt CF4/yr        Emissions|CF4                                  10.869900   
                       Mt CH4/yr        Emissions|CH4                                 388.072796   
                       Mt CO/yr         Emissions|CO                                  934.349885   
                       Mt CO2/yr        Emis

Will execute run for each entry in the `runs_input` list (for loop). Here just taking the first entry as example.

In [253]:
current_run = runs_input[0]
current_run_cfgs = current_run['cfgs']
current_run_scenario_data = current_run['scenariodata']
current_run_output_variables = current_run['output_variables']

# Hector Wrapper
Here, we would create a Hector Wrapper Object which allows us to call the Hector executable after generating input files for the given scenario data and configurations.

### Setting paths and file names
TODO: Update Paths

In [254]:
# Current run details
curent_model = current_run_scenario_data.index[0][0]
curent_region = current_run_scenario_data.index[0][1]
curent_scenario = current_run_scenario_data.index[0][2]

# Setting path to directory for scenario specific temp data
run_dir = '../src/openscm_runner/adapters/hector_adapter/input/run_dir'

# Setting path to input data directory
input_dir = '../src/openscm_runner/adapters/hector_adapter/input'

# Current Run Emissions File Name
cur_run_emis_fn = f'{curent_model}_{curent_region}_{curent_scenario}_emis.csv'

# Current Run .ini File Name
cur_run_ini_fn = f'{curent_model}_{curent_region}_{curent_scenario}_cfg.ini'

# Historical Emissions File Name
hist_emis_fn = f'default_emiss-constraints.csv'

# Default .ini File Name
default_ini_fn = f'default_config.ini'


In [255]:
available_keys_mapping = {
    'ffi_emissions': ['simpleNbox'],
    'luc_emissions': ['simpleNbox'],
    'daccs_uptake': ['simpleNbox'],
    'luc_uptake': ['simpleNbox'],
    'BC_emissions': ['bc'],
    'C2F6_constrain': None,
    'C2F6_emissions': ['C2F6_halocarbon'],
    'CCl4_constrain': None,
    'CCl4_emissions': ['CCl4_halocarbon'],
    'CF4_constrain': None,
    'CF4_emissions': ['CF4_halocarbon'],
    'CFC113_constrain': None,
    'CFC113_emissions': ['CFC113_halocarbon'],
    'CFC114_constrain': None,
    'CFC114_emissions': ['CFC114_halocarbon'],
    'CFC115_constrain': None,
    'CFC115_emissions': ['CFC115_halocarbon'],
    'CFC11_constrain': None,
    'CFC11_emissions': ['CFC11_halocarbon'],
    'CFC12_constrain': None,
    'CFC12_emissions': ['CFC12_halocarbon'],
    'CH3Br_constrain': None,
    'CH3Br_emissions': ['CH3Br_halocarbon'],
    'CH3CCl3_emissions': ['CH3CCl3_halocarbon'],
    'CH3Cl_constrain': None,
    'CH3Cl_emissions': ['CH3Cl_halocarbon'],
    'CH4_constrain': None,
    'CH4_emissions': ['CH4'],
    'CO2_constrain': None,
    'CO_emissions': ['OH', 'ozone'],
    'HCFC141b_constrain': None,
    'HCFC141b_emissions': ['HCFC141b_halocarbon'],
    'HCFC142b_constrain': None,
    'HCFC142b_emissions': ['HCFC142b_halocarbon'],
    'HCFC22_constrain': None,
    'HCFC22_emissions': ['HCFC22_halocarbon'],
    'HFC125_constrain': None,
    'HFC125_emissions': ['HFC125_halocarbon'],
    'HFC134a_constrain': None,
    'HFC134a_emissions': ['HFC134a_halocarbon'],
    'HFC143a_constrain': None,
    'HFC143a_emissions': ['HFC143a_halocarbon'],
    'HFC227ea_constrain': None,
    'HFC227ea_emissions': ['HFC227ea_halocarbon'],
    'HFC23_constrain': None,
    'HFC23_emissions': ['HFC23_halocarbon'],
    'HFC245_constrain': None,
    'HFC245fa_emissions': ['HFC245fa_halocarbon'],
    'HFC32_constrain': None,
    'HFC32_emissions': ['HFC23_halocarbon'],
    'HFC365_constrain': None,
    'HFC365_emissions': None,
    'HFC4310_constrain': None,
    'HFC4310_emissions': ['HFC4310_halocarbon'],
    'N2O_constrain': None,
    'N2O_emissions': ['N2O'],
    'NH3_emissions': ['nh3'],
    'NMVOC_emissions': ['OH', 'ozone'],
    'NOX_emissions': ['OH', 'ozone'],
    'OC_emissions': ['oc'],
    'RF_albedo': ['simpleNbox'],
    'SF6_constrain': None,
    'SF6_emissions': ['SF6_halocarbon'],
    'SO2_emissions': ['so2'],
    'SV': ['so2'],
    'halon1211_constrain': None,
    'halon1211_emissions': ['halon1211_halocarbon'],
    'halon1301_constrain': None,
    'halon1301_emissions': ['halon1301_halocarbon'],
    'halon2402_constrain': None,
    'halon2402_emissions': ['halon2402_halocarbon']
}

### Create Scenario Specific Emissions File

In [256]:
current_run_scenario_data

time                                                                                 2015-01-01  \
model region scenario unit             variable                                                   
IMAGE World  ssp119   Mt BC/yr         Emissions|BC                                    9.727424   
                      kt C2F6/yr       Emissions|C2F6                                  1.570000   
                      kt C6F14/yr      Emissions|C6F14                                 0.350000   
                      kt CF4/yr        Emissions|CF4                                  10.869900   
                      Mt CH4/yr        Emissions|CH4                                 388.072796   
                      Mt CO/yr         Emissions|CO                                  934.349885   
                      Mt CO2/yr        Emissions|CO2|MAGICC AFOLU                   3517.440000   
                                       Emissions|CO2|MAGICC Fossil and Industrial  35635.286298   
                      kt HFC125/yr     Emissions|HFC125                               78.790600   
                      kt HFC134a/yr    Emissions|HFC134a                             201.953400   
                      kt HFC143a/yr    Emissions|HFC143a                              32.381000   
                      kt HFC227ea/yr   Emissions|HFC227ea                              3.663300   
                      kt HFC23/yr      Emissions|HFC23                                14.485700   
                      kt HFC245fa/yr   Emissions|HFC245fa                             11.728500   
                      kt HFC32/yr      Emissions|HFC32                                38.579200   
                      kt HFC4310mee/yr Emissions|HFC4310mee                            1.151600   
                      kt N2O/yr        Emissions|N2O                               10900.000000   
                      Mt NH3/yr        Emissions|NH3                                  65.279703   
                      Mt NOx/yr        Emissions|NOx                                 155.520450   
                      Mt OC/yr         Emissions|OC                                   34.746024   
                      kt SF6/yr        Emissions|SF6                                   8.020000   
                      Mt SO2/yr        Emissions|Sulfur                              100.771167   
                      Mt VOC/yr        Emissions|VOC                                 227.244819   

time                                                                                 2020-01-01  \
model region scenario unit             variable                                                   
IMAGE World  ssp119   Mt BC/yr         Emissions|BC                                    7.824349   
                      kt C2F6/yr       Emissions|C2F6                                  1.952300   
                      kt C6F14/yr      Emissions|C6F14                                 0.435230   
                      kt CF4/yr        Emissions|CF4                                  12.480285   
                      Mt CH4/yr        Emissions|CH4                                 358.907755   
                      Mt CO/yr         Emissions|CO                                  794.367791   
                      Mt CO2/yr        Emissions|CO2|MAGICC AFOLU                   3175.597482   
                                       Emissions|CO2|MAGICC Fossil and Industrial  36518.128970   
                      kt HFC125/yr     Emissions|HFC125                              165.015100   
                      kt HFC134a/yr    Emissions|HFC134a                             249.567200   
                      kt HFC143a/yr    Emissions|HFC143a                              60.100300   
                      kt HFC227ea/yr   Emissions|HFC227ea                              5.300522   
                      kt HFC23/yr      Emissions|HFC23                                 2.516600   
                      kt HFC245fa/yr   

In [257]:
long_scenario_data = pd.melt(current_run_scenario_data, ignore_index = False).reset_index()
long_scenario_data['Date'] = [x.year for x in long_scenario_data.time]
long_scenario_data

,model,region,scenario,unit,variable,time,value,Date
0,IMAGE,World,ssp119,Mt BC/yr,Emissions|BC,2015-01-01,9.727424,2015
1,IMAGE,World,ssp119,kt C2F6/yr,Emissions|C2F6,2015-01-01,1.570000,2015
2,IMAGE,World,ssp119,kt C6F14/yr,Emissions|C6F14,2015-01-01,0.350000,2015
3,IMAGE,World,ssp119,kt CF4/yr,Emissions|CF4,2015-01-01,10.869900,2015
4,IMAGE,World,ssp119,Mt CH4/yr,Emissions|CH4,2015-01-01,388.072796,2015
...,...,...,...,...,...,...,...,...
225,IMAGE,World,ssp119,Mt NOx/yr,Emissions|NOx,2100-01-01,37.368243,2100
226,IMAGE,World,ssp119,Mt OC/yr,Emissions|OC,2100-01-01,12.939701,2100
227,IMAGE,World,ssp119,kt SF6/yr,Emissions|SF6,2100-01-01,0.896800,2100
228,IMAGE,World,ssp119,Mt SO2/yr,Emissions|Sulfur,2100-01-01,9.461409,2100


In [258]:
# Filter out data not available for use in Hector
# And update variable names to Hector variable names
variables = long_scenario_data.reset_index()['variable']
keep_vec = [True] * len(variables)
new_variable_names = [None] * len(variables)
for i, var in enumerate(variables):
    split_variable = var.split('|')
    if len(split_variable) == 2:
        em = split_variable[1]
        variable_type = split_variable[0].lower()
        new_variable_name = f'{em}_{variable_type}'
    else:
        new_variable_name = None
    # print(f'{var} : {new_variable_name} : {new_variable_name in available_keys_mapping.keys()}')
    keep_vec[i] = (new_variable_name in available_keys_mapping.keys())
    new_variable_names[i] = new_variable_name

long_scenario_hector_data = long_scenario_data.copy()
long_scenario_hector_data['variable'] = new_variable_names
long_scenario_hector_data = long_scenario_hector_data[keep_vec]
long_scenario_hector_data

,model,region,scenario,unit,variable,time,value,Date
0,IMAGE,World,ssp119,Mt BC/yr,BC_emissions,2015-01-01,9.727424,2015
1,IMAGE,World,ssp119,kt C2F6/yr,C2F6_emissions,2015-01-01,1.570000,2015
3,IMAGE,World,ssp119,kt CF4/yr,CF4_emissions,2015-01-01,10.869900,2015
4,IMAGE,World,ssp119,Mt CH4/yr,CH4_emissions,2015-01-01,388.072796,2015
5,IMAGE,World,ssp119,Mt CO/yr,CO_emissions,2015-01-01,934.349885,2015
...,...,...,...,...,...,...,...,...
221,IMAGE,World,ssp119,kt HFC32/yr,HFC32_emissions,2100-01-01,0.000000,2100
223,IMAGE,World,ssp119,kt N2O/yr,N2O_emissions,2100-01-01,8339.071915,2100
224,IMAGE,World,ssp119,Mt NH3/yr,NH3_emissions,2100-01-01,63.182134,2100
226,IMAGE,World,ssp119,Mt OC/yr,OC_emissions,2100-01-01,12.939701,2100


In [259]:
hector_format_data = long_scenario_hector_data.pivot(index=['Date'], columns=['unit', 'variable'], values='value')
hector_format_data = hector_format_data.reset_index(col_level='variable')
hector_format_data

unit,,Mt BC/yr,kt C2F6/yr,kt CF4/yr,Mt CH4/yr,Mt CO/yr,kt HFC125/yr,kt HFC134a/yr,kt HFC143a/yr,kt HFC227ea/yr,kt HFC23/yr,kt HFC245fa/yr,kt HFC32/yr,kt N2O/yr,Mt NH3/yr,Mt OC/yr,kt SF6/yr
variable,Date,BC_emissions,C2F6_emissions,CF4_emissions,CH4_emissions,CO_emissions,HFC125_emissions,HFC134a_emissions,HFC143a_emissions,HFC227ea_emissions,HFC23_emissions,HFC245fa_emissions,HFC32_emissions,N2O_emissions,NH3_emissions,OC_emissions,SF6_emissions
0,2015,9.727424,1.5700,10.869900,388.072796,934.349885,78.790600,201.9534,32.3810,3.663300,14.4857,11.728500,38.5792,10900.000000,65.279703,34.746024,8.0200
1,2020,7.824349,1.9523,12.480285,358.907755,794.367791,165.015100,249.5672,60.1003,5.300522,2.5166,20.189008,6.7024,10706.384825,65.138289,29.146213,6.5973
2,2030,4.434228,0.4605,4.118062,243.027020,526.459260,33.340622,31.2569,11.9486,0.796542,0.0828,3.961356,0.2205,8393.759091,64.858355,20.816355,3.0909
3,2040,3.545660,0.2826,3.169171,200.612862,515.611615,18.238580,13.9191,6.7780,0.367985,0.0230,2.271800,0.0613,8376.573664,65.217184,20.641517,2.7046
4,2050,2.797528,0.1657,2.300779,170.196862,446.510359,20.687888,14.7522,7.8410,0.383938,0.0118,2.707863,0.0314,8073.585197,64.732143,17.392083,2.2198
5,2060,2.518834,0.1013,1.854183,157.558897,415.535958,22.315640,15.1540,8.5067,0.361748,0.0063,3.042408,0.0168,7941.611765,64.102649,16.219916,1.9331
6,2070,2.370297,0.0659,1.619485,146.459704,396.223686,24.044969,16.0660,9.3192,0.345316,0.0033,3.398962,0.0088,7958.151734,63.874616,15.568949,1.6611
7,2080,2.243716,0.0450,1.425487,132.247397,374.055036,25.522914,17.4072,9.9967,0.334761,0.0017,3.724445,0.0045,7978.923965,63.614903,14.532016,1.3755
8,2090,2.177403,0.0273,1.175089,120.764306,362.739212,26.088560,18.5388,10.2095,0.319768,0.0008,3.929207,0.0021,8204.736256,63.518249,13.966907,1.1400


In [260]:
# Save Data
curent_model = current_run_scenario_data.index[0][0]
curent_region = current_run_scenario_data.index[0][1]
curent_scenario = current_run_scenario_data.index[0][2]
hector_format_data.to_csv(os.path.join(run_dir, cur_run_emis_fn))

In [261]:
hector_format_data

unit,,Mt BC/yr,kt C2F6/yr,kt CF4/yr,Mt CH4/yr,Mt CO/yr,kt HFC125/yr,kt HFC134a/yr,kt HFC143a/yr,kt HFC227ea/yr,kt HFC23/yr,kt HFC245fa/yr,kt HFC32/yr,kt N2O/yr,Mt NH3/yr,Mt OC/yr,kt SF6/yr
variable,Date,BC_emissions,C2F6_emissions,CF4_emissions,CH4_emissions,CO_emissions,HFC125_emissions,HFC134a_emissions,HFC143a_emissions,HFC227ea_emissions,HFC23_emissions,HFC245fa_emissions,HFC32_emissions,N2O_emissions,NH3_emissions,OC_emissions,SF6_emissions
0,2015,9.727424,1.5700,10.869900,388.072796,934.349885,78.790600,201.9534,32.3810,3.663300,14.4857,11.728500,38.5792,10900.000000,65.279703,34.746024,8.0200
1,2020,7.824349,1.9523,12.480285,358.907755,794.367791,165.015100,249.5672,60.1003,5.300522,2.5166,20.189008,6.7024,10706.384825,65.138289,29.146213,6.5973
2,2030,4.434228,0.4605,4.118062,243.027020,526.459260,33.340622,31.2569,11.9486,0.796542,0.0828,3.961356,0.2205,8393.759091,64.858355,20.816355,3.0909
3,2040,3.545660,0.2826,3.169171,200.612862,515.611615,18.238580,13.9191,6.7780,0.367985,0.0230,2.271800,0.0613,8376.573664,65.217184,20.641517,2.7046
4,2050,2.797528,0.1657,2.300779,170.196862,446.510359,20.687888,14.7522,7.8410,0.383938,0.0118,2.707863,0.0314,8073.585197,64.732143,17.392083,2.2198
5,2060,2.518834,0.1013,1.854183,157.558897,415.535958,22.315640,15.1540,8.5067,0.361748,0.0063,3.042408,0.0168,7941.611765,64.102649,16.219916,1.9331
6,2070,2.370297,0.0659,1.619485,146.459704,396.223686,24.044969,16.0660,9.3192,0.345316,0.0033,3.398962,0.0088,7958.151734,63.874616,15.568949,1.6611
7,2080,2.243716,0.0450,1.425487,132.247397,374.055036,25.522914,17.4072,9.9967,0.334761,0.0017,3.724445,0.0045,7978.923965,63.614903,14.532016,1.3755
8,2090,2.177403,0.0273,1.175089,120.764306,362.739212,26.088560,18.5388,10.2095,0.319768,0.0008,3.929207,0.0021,8204.736256,63.518249,13.966907,1.1400


### Create Scenario/Configuration Specific .ini File

#### TODO:

* Emission paths / years
* [simpleNbox]
    * beta
* [temperature]
    * S
    * diff

In [267]:
# Read default ini file
config_parser = ConfigParser(inline_comment_prefixes = (";",))
config_parser.read(os.path.join(input_dir, default_ini_fn))

['/Users/prim232/gitrepos/openscm-runner/src/openscm_runner/adapters/hector_adapter/input/hector_template.ini']

In [268]:
# If config value is provided, update value
for key, value in current_run_cfgs.items():
    if(key == 'beta'):
        config_parser.set('simpleNbox', 'beta', str(value))
    elif(key == 'S'):
        config_parser.set('temperature', 'S', str(value))
    elif(key == 'diff'):
        config_parser.set('temperature', 'diff', str(value))
    else:
        next

In [269]:
int(hector_format_data.iloc[0].iloc[0])

2015

In [270]:
start_year = int(hector_format_data.iloc[0].iloc[0])
# For each time series in the input data, check if it's data that can be pointed to
# in the ini file (which may actually appear in multiple places).
# If so, add a pointer to the time series in each place it shows up, setting the ini
# to switch to that time series beginning at the first year of the input data
for muticol in hector_format_data.columns:
    col = muticol[1]
    if col in available_keys_mapping.keys():
        sections = available_keys_mapping[col]
        if sections is not None:
            for section in sections:
                print(f'section: {section}, col: {col}')
                config_parser.set(
                    section, 
                    f'{col}[{start_year}]', 
                    f'csv:{cur_run_emis_fn}')

section: bc, col: BC_emissions
section: C2F6_halocarbon, col: C2F6_emissions
section: CF4_halocarbon, col: CF4_emissions
section: CH4, col: CH4_emissions
section: OH, col: CO_emissions
section: ozone, col: CO_emissions
section: HFC125_halocarbon, col: HFC125_emissions
section: HFC134a_halocarbon, col: HFC134a_emissions
section: HFC143a_halocarbon, col: HFC143a_emissions
section: HFC227ea_halocarbon, col: HFC227ea_emissions
section: HFC23_halocarbon, col: HFC23_emissions
section: HFC245fa_halocarbon, col: HFC245fa_emissions
section: HFC23_halocarbon, col: HFC32_emissions
section: N2O, col: N2O_emissions
section: nh3, col: NH3_emissions
section: oc, col: OC_emissions
section: SF6_halocarbon, col: SF6_emissions


In [271]:
with open(os.path.join(run_dir, cur_run_ini_fn), 'w') as configfile:
    config_parser.write(configfile)